In [1]:
import os
from chains import create_assistant_chain, create_input_history_enricher_chain, create_detail_retriever, create_element_extractor_chain, router
from langchain_openai import ChatOpenAI

from agents.orchestrator import orchestrator_graph
from conversation.local.utils import init_local_history
from conversation.remote.utils import retrieve_remote_history, update_remote_history
from utils import print_heading


llm = ChatOpenAI(
    model='gpt-4o-mini', 
    api_key=os.getenv('OPENAI_API_KEY')
)

agent_tools = {
    'assistant_chain': create_assistant_chain(llm),
    'input_history_enricher_chain': create_input_history_enricher_chain(llm),
    'get_element_in_text': create_element_extractor_chain(llm),
    'detail_retriever': create_detail_retriever(llm),
    'router': router(llm)
}


async def main(user_id: str):
    data = retrieve_remote_history(user_id)
    history = init_local_history(data, llm)

    constant_state = {
        'user_id': user_id,
        'agent_tools': agent_tools,
        'history': history,
        'llm': llm,
        'characteristics': dict(),
        'current_agent': None,
        'token_processor': {
            'is_async': False, 
            'fn': lambda output, token: print(token, end=''),
        }
    }

    state = {
        **constant_state,
        'input': None
    }

    while True:
        user_input = input()
        if user_input.lower() == 'bye':
            break
        elif user_input.lower() == 'r':
            print(f'Ejecución de agente {state['current_agent']} terminada. Ingrese su nueva petición.')

            state['current_agent'] = None
            user_input = input()
        
        print_heading('User')
        print(user_input)

        state['input'] = user_input
        
        print_heading('Maya')
        state = await orchestrator_graph.ainvoke(state)

        new_messages = {'input': state['input'], 'output': state['output']}

        update_remote_history(
            user_id=user_id, new_messages=new_messages,
        )
        
    update_remote_history(
        user_id=user_id, new_summary=history.moving_summary_buffer,
        new_last_message=len(history.chat_memory.messages),
    )
    return state

In [2]:
await main('2')

/Users/brunoramirezdelangel/Desktop/hackathon/conversation/local/utils.py:41: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


=================================================  User  =================================================
Hola
=================================================  Maya  =================================================
¡Hola! Soy Maya, tu asistente virtual de Banorte. ¿En qué puedo ayudarte hoy?
=================================================  User  =================================================
Quiero automatizar un pago
=================================================  Maya  =================================================
¡Claro! Para ayudarte a automatizar un pago, necesitaré algunos detalles. Por favor, indícame lo siguiente:

1. **Tipo de movimiento**: ¿Es un ingreso o un egreso?
2. **Frecuencia de transacción**: ¿Con qué frecuencia deseas que se realice este pago? (por ejemplo, semanal, mensual, etc.)
3. **Cantidad a mover**: ¿Cuál es el monto que deseas automatizar?

Con esta información, podré asistirte mejor.
LEAVING ORQ


{'input': 'Quiero automatizar un pago',
 'history_enriched_input': 'Quiero automatizar un pago.',
 'output': '¡Claro! Para ayudarte a automatizar un pago, necesitaré algunos detalles. Por favor, indícame lo siguiente:\n\n1. **Tipo de movimiento**: ¿Es un ingreso o un egreso?\n2. **Frecuencia de transacción**: ¿Con qué frecuencia deseas que se realice este pago? (por ejemplo, semanal, mensual, etc.)\n3. **Cantidad a mover**: ¿Cuál es el monto que deseas automatizar?\n\nCon esta información, podré asistirte mejor.',
 'history': ConversationSummaryBufferMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1210956a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12071f6e0>, root_client=<openai.OpenAI object at 0x117d82ed0>, root_async_client=<openai.AsyncOpenAI object at 0x121095700>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), prompt=PromptTemplate(input_variables=['new_lines', 'summ

In [1]:
from main import main

In [2]:
await main('2')

/Users/brunoramirezdelangel/Desktop/hackathon/conversation/local/utils.py:41: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


=================================================  User  =================================================
Hola
=================================================  Maya  =================================================
¡Hola! Soy Maya, tu asistente del Banco de Banorte. ¿En qué puedo ayudarte hoy? Puedes consultar tus inversiones, automatizar pagos o revisar tu información.
=================================================  User  =================================================
Quiero invertir mi dinero
=================================================  Maya  =================================================
ENTERING INVESTOR
¡Eso suena genial! Para poder ayudarte mejor con la inversión, necesitaré que me proporciones algunos detalles. Por favor, indícame lo siguiente:

1. **Símbolo de la acción (ticker)**: ¿Cuál es el símbolo de la acción en la que deseas invertir?
2. **Número de acciones**: ¿Cuántas acciones te gustaría comprar?
3. **Costo de acciones**: ¿Cuál es el costo por acción

CancelledError: 

In [4]:
len('"')

1

In [5]:
await run_orchestrator('2', llm, agent_tools)

/Users/brunoramirezdelangel/Desktop/hackathon/conversation/local/utils.py:41: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


Hola
¡Hola! Bienvenido al servicio de Banorte. ¿En qué puedo ayudarte hoy? Puedes consultar tus inversiones, automatizar pagos, revisar tu información o cualquier otra duda que tengas.
hola bruh
¡Hola! ¿Cómo estás? Si necesitas ayuda con algo relacionado con tu cuenta o servicios de Banorte, estoy aquí para asistirte. ¿Qué puedo hacer por ti hoy?

Parece que no has escrito nada. Si tienes alguna pregunta o necesitas ayuda con tus inversiones, pagos o información de tu cuenta, no dudes en decírmelo. ¡Estoy aquí para ayudarte!

Parece que no estás escribiendo nada en este momento. Si tienes alguna consulta o necesitas asistencia con algo específico, no dudes en decírmelo. Estoy aquí para ayudarte con tus necesidades en Banorte. ¡Espero tu mensaje!


{'input': '',
 'history_enriched_input': 'Pregunta: ¿Cómo puedo obtener ayuda con mis inversiones, pagos o información de mi cuenta?',
 'output': 'Parece que no estás escribiendo nada en este momento. Si tienes alguna consulta o necesitas asistencia con algo específico, no dudes en decírmelo. Estoy aquí para ayudarte con tus necesidades en Banorte. ¡Espero tu mensaje!',
 'history': ConversationSummaryBufferMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115a76f30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115abc7d0>, root_client=<openai.OpenAI object at 0x1153a04a0>, root_async_client=<openai.AsyncOpenAI object at 0x115a76f60>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), prompt=PromptTemplate(input_variables=['new_lines', 'summary'], input_types={}, partial_variables={}, template='Resume progresivamente las líneas de conversación proporcionadas, añadiendo a la resumen ant